Players dataset creation

The Fantacalcio players list is manually downloaded from https://www.fantacalcio.it/quotazioni-fantacalcio

Here, the players database is generated, by merging the Fantacalcio list to stats downloaded from http://fbref.com

In [1]:
import pandas as pd

Load fbref data for outfield players and goalkeepers.

Generate fbref player list, adding player surname (with special characters replaced to normal ones)

In [2]:
rcsv = pd.read_csv('fbref_data/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [3]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

keepers_ID = len(outfield_players)

players

,player,team
0,Oliver Abildgaard,Hellas Verona
1,Tammy Abraham,Roma
2,Francesco Acerbi,Inter
3,Yacine Adli,Milan
4,Michel Aebischer,Bologna
...,...,...
567,Ciprian Tătărușanu,Milan
568,Pietro Terracciano,Fiorentina
569,Guglielmo Vicario,Empoli
570,Jeroen Zoet,Spezia


In [4]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [5]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [6]:
print(players[['player', 'surname']].to_string())

                         player           surname
0             Oliver Abildgaard        Abildgaard
1                 Tammy Abraham           Abraham
2              Francesco Acerbi            Acerbi
3                   Yacine Adli              Adli
4              Michel Aebischer         Aebischer
5              Felix Afena-Gyan        Afena-Gyan
6                 Kevin Agudelo           Agudelo
7                      Ola Aina              Aina
8                 Emanuel Aiwum             Aiwum
9        Jean-Daniel Akpa-Akpro        Akpa-Akpro
10                 Luis Alberto           Alberto
11     Agustín Álvarez Martínez          Martinez
12                 Kelvin Amian             Amian
13                 Bruno Amione            Amione
14                 Bruno Amione            Amione
15                 Ethan Ampadu            Ampadu
16               Sofyan Amrabat           Amrabat
17              Felipe Anderson          Anderson
18                Janis Antiste           Antiste


Replace the surname for some specific players, according to config/name_fix.txt file.

This is done for players for which the decoded fbref surname doesn't correspond to Fantacalcio list.

In [7]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix



Ostigard
Kim
Hojlund
Gytkjaer
Augusto
Maehle
Kjaer
Djuricic
Djuric
Cabral
Alvarez


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Monza
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


Load players from Fantacalcio list.

In [8]:
fc_data = pd.read_excel('fantacalcio/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players['surname'][i] = spl[-2]
        fc_players['initial'][i] = spl[-1][0]
    else:
        fc_players['surname'][i] = spl[-1]
        fc_players['initial'][i] = ''
    
fc_players



C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2263921821.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-1]
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2263921821.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['initial'][i] = ''
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2263921821.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-

,id,r,name,team,surname,initial
0,572,P,Meret,Napoli,Meret,
1,2814,P,Provedel,Lazio,Provedel,
2,4964,P,Vicario,Empoli,Vicario,
3,453,P,Szczesny,Juventus,Szczesny,
4,2134,P,Falcone,Lecce,Falcone,
...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,
539,5837,A,Voelkerling Persson,Lecce,Persson,
540,6113,A,Montevago,Sampdoria,Montevago,
541,6143,A,Krollis,Spezia,Krollis,


Associate players from Fantacalcio list to ID for FBref data.

In [9]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players['fb_ID'][i] = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players['fb_ID'][i] = j
                
        

C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2200947104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = -1
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2200947104.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


Print players for which the association failed.

Most of them are players who didn't play a single Serie A game this season with their team. If that is the case, and there is data from their previous team, that is taken here.

Others are ones for which the FBRef surname doesn't correspond to Fantacalcio one.


For example, Cabral is Arthur for FBref.

Correction is made in the name_fix code above.

In [10]:
exceptions = ['pellegrini', 'berardi', 'romagnoli'] # exceptions for such players that have the same surname as others (Berardi A., Luca Pellegrini)

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        found = False
        for j in range(players.shape[0]):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                if(not(players['surname'][j].lower() in exceptions)):
                    if((fc_players['r'][i] == 'P') == (j >= keepers_ID)):
                        fc_players['fb_ID'][i] = j
                        found = True
        if(found):
            print(fc_players['name'][i] + ' from previous team stats')
        else:
            print(fc_players['name'][i] + ' not found')

C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\362391242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


Gollini from previous team stats
Mirante not found
Sarr M. not found
Lamanna not found
Ujkani not found
Berisha not found
Marchetti not found
Perilli not found
Padelli not found
Perisan not found
Bardi not found
Cordaz not found
Pinsoglio not found
Fiorillo not found
Cragno not found
Sirigu not found
Cerofolini not found
Rossi F. not found
Ravaglia F. not found
Brancolini not found
Bleve not found
Berardi A. not found
Russo A. not found
Gemello not found
Ravaglia not found
Boer not found
Adamonis not found
Marfella not found
Piana not found
Bagnolini not found
Svilar not found
Sorrentino A. not found
Ciezkowski not found
Saro not found
Vasquez D. not found
Turk not found
Pellegrini Lu. not found
Gravillon not found
Gunter from previous team stats
Ceccaroni not found
Bereszynski from previous team stats
Aiwu not found
Zeefuik not found
Romagnoli S. not found
Tonelli not found
Radu from previous team stats
Paletta not found
Fares not found
Romagna not found
Cassandro not found
Amey not f

In [11]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,572,P,Meret,Napoli,Meret,,551
1,2814,P,Provedel,Lazio,Provedel,,560
2,4964,P,Vicario,Empoli,Vicario,,569
3,453,P,Szczesny,Juventus,Szczesny,,566
4,2134,P,Falcone,Lecce,Falcone,,546
...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,134
539,5837,A,Voelkerling Persson,Lecce,Persson,,508
540,6113,A,Montevago,Sampdoria,Montevago,,331
541,6143,A,Krollis,Spezia,Krollis,,258


Populate players dataset with stats from FBref, for outfield players and goalkeepers

In [12]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            

C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

In [13]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [14]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]

C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_16864\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [15]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,572,P,Meret,Napoli,Meret,,551,25-325,1997,0,...,26.7,128,19.5,27.1,221,6,2.7,24,1.14,17.2
1,2814,P,Provedel,Lazio,Provedel,,560,28-330,1994,0,...,33.0,120,34.2,34.5,279,7,2.5,39,1.86,18.2
2,4964,P,Vicario,Empoli,Vicario,,569,26-126,1996,0,...,34.0,108,47.2,42.4,426,25,5.9,12,0.57,10.9
3,453,P,Szczesny,Juventus,Szczesny,,566,32-298,1990,0,...,35.0,72,41.7,38.7,178,4,2.2,12,0.83,15.8
4,2134,P,Falcone,Lecce,Falcone,,546,27-304,1995,0,...,41.3,163,76.1,51.7,316,15,4.7,20,0.95,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,134,24-208,1998,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
539,5837,A,Voelkerling Persson,Lecce,Persson,,508,20-026,2003,3,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
540,6113,A,Montevago,Sampdoria,Montevago,,331,19-329,2003,6,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
541,6143,A,Krollis,Spezia,Krollis,,258,21-105,2001,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Load votes database, to add data to players database (mean vote and its standard deviation)

In [16]:
import numpy as np

votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)

Compute the average Serie A Goal Keeper mean vote and vote std

In [17]:
min_votes = 6

perf_df_P = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]): 
    if(fc_players.loc[i]['r'] == 'P'):
        v = np.array([])
        for j in range(votes.shape[0]):
            if(fc_players['name'][i] == votes['player'][j]):
                v = np.append(v, votes['vote'][j])

        if(v.shape[0] >= min_votes - 1):
            row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df_P.columns)
            perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


print(perf_df_P.mean())

perf_df_P


vote_avg    6.211716
vote_std    0.441335
dtype: float64


,vote_avg,vote_std
0,6.261905,0.478566
1,6.261905,0.365769
2,6.476190,0.392677
3,6.033333,0.339935
4,6.309524,0.361089
5,6.285714,0.424585
6,6.000000,0.436436
7,6.115385,0.287820
8,6.366667,0.426875
9,6.142857,0.412393


Add to players data their mean vote (and its standard deviation).

For players who don't have a minimum amount of games, more data to reach this value is computed, according to the average Serie A player vote (and std). For goalkeepers, this values are different.


In [18]:
min_votes = 6

#outfield players
mean_def = 6
std_def = 0.58

#goalkeepers
mean_def_P = 6.22
std_def_P = 0.43


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    mean_def_i = mean_def
    std_def_i = std_def
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

,vote_avg,vote_std
0,6.261905,0.478566
1,6.261905,0.365769
2,6.476190,0.392677
3,6.033333,0.339935
4,6.309524,0.361089
...,...,...
538,5.947397,0.490100
539,5.694209,0.383547
540,5.623932,0.280948
541,6.124287,0.483769


In [19]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,572,P,Meret,Napoli,Meret,,551,25-325,1997,0,...,19.5,27.1,221,6,2.7,24,1.14,17.2,6.261905,0.478566
1,2814,P,Provedel,Lazio,Provedel,,560,28-330,1994,0,...,34.2,34.5,279,7,2.5,39,1.86,18.2,6.261905,0.365769
2,4964,P,Vicario,Empoli,Vicario,,569,26-126,1996,0,...,47.2,42.4,426,25,5.9,12,0.57,10.9,6.476190,0.392677
3,453,P,Szczesny,Juventus,Szczesny,,566,32-298,1990,0,...,41.7,38.7,178,4,2.2,12,0.83,15.8,6.033333,0.339935
4,2134,P,Falcone,Lecce,Falcone,,546,27-304,1995,0,...,76.1,51.7,316,15,4.7,20,0.95,12.5,6.309524,0.361089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,134,24-208,1998,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.947397,0.490100
539,5837,A,Voelkerling Persson,Lecce,Persson,,508,20-026,2003,3,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.694209,0.383547
540,6113,A,Montevago,Sampdoria,Montevago,,331,19-329,2003,6,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.623932,0.280948
541,6143,A,Krollis,Spezia,Krollis,,258,21-105,2001,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.124287,0.483769


In [20]:
fc_players.columns[123]

'gk_shots_on_target_against'

For goalkeepers who didn't play a miminum amount of games, data is weightly averaged with one of the main goalkeeper of their same team.

In [21]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Zoet, 0.5
Pegolo, 0.33333333333333337
Gollini, 0.5
Mirante, 0.0
Sarr M., 0.0
Lamanna, 0.0
Ujkani, 0.0
Berisha, 0.0
Marchetti, 0.0
Perilli, 0.0
Padelli, 0.0
Perisan, 0.0
Bardi, 0.0
Cordaz, 0.0
Pinsoglio, 0.0
Fiorillo, 0.0
Cragno, 0.0
Sirigu, 0.0
Cerofolini, 0.0
Rossi F., 0.0
Ravaglia F., 0.0
Brancolini, 0.0
Bleve, 0.0
Berardi A., 0.0
Russo A., 0.0
Gemello, 0.0
Ravaglia, 0.0
Boer, 0.0
Adamonis, 0.0
Marfella, 0.0
Zovko, 0.16666666666666663
Piana, 0.0
Bagnolini, 0.0
Luis Maximiano, 0.16666666666666663
Svilar, 0.0
Sorrentino A., 0.0
Ciezkowski, 0.0
Saro, 0.0
Vasquez D., 0.0
Turk, 0.0


In [22]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,572,P,Meret,Napoli,Meret,,551,25-325,1997,0,...,19.5,27.1,221.0,6.0,2.7,24.0,1.14,17.2,6.261905,0.478566
1,2814,P,Provedel,Lazio,Provedel,,560,28-330,1994,0,...,34.2,34.5,279.0,7.0,2.5,39.0,1.86,18.2,6.261905,0.365769
2,4964,P,Vicario,Empoli,Vicario,,569,26-126,1996,0,...,47.2,42.4,426.0,25.0,5.9,12.0,0.57,10.9,6.476190,0.392677
3,453,P,Szczesny,Juventus,Szczesny,,566,32-298,1990,0,...,41.7,38.7,178.0,4.0,2.2,12.0,0.83,15.8,6.033333,0.339935
4,2134,P,Falcone,Lecce,Falcone,,546,27-304,1995,0,...,76.1,51.7,316.0,15.0,4.7,20.0,0.95,12.5,6.309524,0.361089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,134,24-208,1998,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.947397,0.490100
539,5837,A,Voelkerling Persson,Lecce,Persson,,508,20-026,2003,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.694209,0.383547
540,6113,A,Montevago,Sampdoria,Montevago,,331,19-329,2003,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.623932,0.280948
541,6143,A,Krollis,Spezia,Krollis,,258,21-105,2001,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.124287,0.483769


Save to file.

In [23]:
fc_players.to_excel('mid_outputs/players_stats.xlsx')